In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [3]:
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train_data.head()

In [4]:
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test_data.head()

In [5]:
sample_submission_data = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
sample_submission_data.head()

In [6]:
X_train = train_data.iloc[:, [1,2,4,5,6,7,8,9,10,11]]
y_train = train_data['Transported']
new_X_train = pd.get_dummies(X_train)
print(new_X_train)

In [7]:
new_X_train_copy = new_X_train.copy()
#new_y_train_copy = new_y_train.copy()

cols_with_missing = (col for col in new_X_train_copy.columns 
                                 if new_X_train_copy[col].isnull().any())
for col in cols_with_missing:
    new_X_train_copy[col + '_was_missing'] = new_X_train_copy[col].isnull()

my_imputer = SimpleImputer()
new_X_train_copy = pd.DataFrame(my_imputer.fit_transform(new_X_train))
new_X_train_copy.columns = new_X_train.columns
print(new_X_train_copy)

In [8]:
model = RandomForestClassifier()

In [9]:
model.fit(new_X_train_copy, y_train)

In [10]:
X_train_prediction = model.predict(new_X_train_copy)
training_data_accuracy = accuracy_score(X_train_prediction, y_train) #compare y-hat and y for training data
print(X_train_prediction)
print('Accuracy on training data : ', training_data_accuracy)

In [11]:
X_test = test_data.iloc[:, [1,2,4,5,6,7,8,9,10,11]]
new_X_test = pd.get_dummies(X_test)
new_X_test_copy = new_X_test.copy()

cols_with_missing = (col for col in new_X_test_copy.columns 
                                 if new_X_test_copy[col].isnull().any())
for col in cols_with_missing:
    new_X_test_copy[col + '_was_missing'] = new_X_test_copy[col].isnull()

my_imputer = SimpleImputer()
new_X_test_copy = pd.DataFrame(my_imputer.fit_transform(new_X_test))
new_X_test_copy.columns = new_X_test.columns

predictions = model.predict(new_X_test_copy)



output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")